In [4]:
from scholia import query
import sqlite3
import re
import networkx as nx
from itertools import combinations
test = query.doi_to_qs("10.1371/journal.pone.0147152")

print(test)

['Q30386191']


In [42]:
db = sqlite3.connect("ARCSLiterature.sqlite3")
cursor = db.cursor()
dbquery = cursor.execute('SELECT DOI, AuthorKeywords FROM bibliography;')

doi_q_dict = {}
counter = 0

for d in dbquery:
    for x in d:
        if d[0]:
            try:
                if d[0] not in doi_q_dict:
                    doi_q_dict[d[0]] = query.doi_to_qs(d[0])[0]
                    counter += 1
                    print(str(counter))
                else:
                    continue
            except IndexError:
                continue

        #print(("*" * 20))
        #print(x)
        #match = re.findall("citizen science", x, re.IGNORECASE)
        #if match:
        #    print(query.doi_to_qs(d[0]), match)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [36]:
G = nx.Graph()

dbquery = cursor.execute('SELECT DOI, Keywords FROM bibliography;')
for d in dbquery:
    keywordsperarticle = d[1].split("; ")
    #print("---")
    combos = combinations(keywordsperarticle, 2)
    for c in combos:
        G.add_edge(c[0].lower(), c[1].lower())
        print(c[0].lower(), c[1].lower())
        
nx.write_gexf(G, "keywordcombinations.gexf")

citizen science american pika
citizen science distribution
citizen science glacier national park
citizen science observer variability
citizen science ochotona princeps
citizen science professional
citizen science volunteer
citizen science newtech
american pika distribution
american pika glacier national park
american pika observer variability
american pika ochotona princeps
american pika professional
american pika volunteer
american pika newtech
distribution glacier national park
distribution observer variability
distribution ochotona princeps
distribution professional
distribution volunteer
distribution newtech
glacier national park observer variability
glacier national park ochotona princeps
glacier national park professional
glacier national park volunteer
glacier national park newtech
observer variability ochotona princeps
observer variability professional
observer variability volunteer
observer variability newtech
ochotona princeps professional
ochotona princeps volunteer
ochotona

In [43]:
import pandas as pd

dfDoiQ = pd.DataFrame.from_dict(doi_q_dict, orient='index')
dfDoiQ.to_csv("doi_q.csv")


In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """#defaultView:Graph
SELECT ?node ?nodeLabel ?nodeImage ?childNode ?childNodeLabel ?childNodeImage ?rgb WHERE {
  {
    BIND(wd:Q1093434 AS ?node)
    ?node ?p ?i.
    OPTIONAL { ?node wdt:P18 ?nodeImage. }
    ?childNode ?x ?p.
    ?childNode rdf:type wikibase:Property.
    FILTER(STRSTARTS(STR(?i), "http://www.wikidata.org/entity/Q"))
    FILTER(STRSTARTS(STR(?childNode), "http://www.wikidata.org/entity/P"))
  }
  UNION
  {
    BIND("EFFBD8" AS ?rgb)
    wd:Q1093434 ?p ?childNode.
    OPTIONAL { ?childNode wdt:P18 ?childNodeImage. }
    ?node ?x ?p.
    ?node rdf:type wikibase:Property.
    FILTER(STRSTARTS(STR(?childNode), "http://www.wikidata.org/entity/Q"))
  }
  OPTIONAL {
    ?node wdt:P18 ?nodeImage.
    ?childNode wdt:P18 ?childNodeImage.
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}"""


def get_results(endpoint_url, query):
    sparql = SPARQLWrapper(endpoint_url)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'childNode': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q8379395'}, 'node': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P910'}, 'rgb': {'type': 'literal', 'value': 'EFFBD8'}, 'nodeLabel': {'xml:lang': 'en', 'type': 'literal', 'value': "topic's main category"}, 'childNodeLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Category:Citizen science'}}
{'childNode': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P279'}, 'node': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1093434'}, 'nodeLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'citizen science'}, 'childNodeLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'subclass of'}, 'nodeImage': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Scanning%20the%20cliffs%20near%20Logan%20Pass%20for%20mountain%20goats%20%28Citizen%20Science%29%20%284427399123%29.jpg'}}
{'childNode': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/P910'}, 'node': 